**1. Geliştirme Ortamı (Kurulum)**

Proje için gerekli tüm kütüphaneler yüklenmektedir.
LangChain, FAISS, Gemini API ve Gradio bileşenleri RAG tabanlı chatbot için gereklidir.

In [3]:
!pip install -q google-generativeai langchain langchain-community faiss-cpu sentence-transformers langchain-google-genai datasets huggingface_hub pyarrow gradio


**2. API ve Modül Kurulumu**

Bu hücrede gerekli modüller import edilmekte ve Google Gemini ile Hugging Face API anahtarları tanımlanmaktadır.
API’ler, modelin metin üretimi ve embedding oluşturması için kullanılır.

In [4]:
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from datasets import load_dataset
from google.colab import userdata
import gradio as gr
import os

try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    HF_TOKEN = userdata.get('HF_TOKEN')
except:
    print("Secrets bulunamadı")

os.environ['HUGGINGFACE_HUB_TOKEN'] = HF_TOKEN
genai.configure(api_key=GOOGLE_API_KEY)


**3. Veri Seti Hazırlama**

Bu hücrede “aliarda/Turkish-Poems-19K” adlı Türkçe şiir veri seti Hugging Face’ten yüklenmektedir.
siir_metni sütunundaki ilk 1000 şiir alınarak modelin öğrenmesi için kullanılmaktadır.

In [5]:
dataset = load_dataset("aliarda/Turkish-Poems-19K", token=HF_TOKEN)
print(f"Veri seti yüklendi! Toplam {len(dataset['train'])} şiir bulundu.")

poems = [
    item["siir_metni"].strip()
    for item in dataset["train"]
    if item["siir_metni"] and isinstance(item["siir_metni"], str) and item["siir_metni"].strip()
]
poems = poems[:1000]
print(f"{len(poems)} şiir işleme alındı.")

README.md:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

tum_siirler.parquet:   0%|          | 0.00/10.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18926 [00:00<?, ? examples/s]

Veri seti yüklendi! Toplam 18926 şiir bulundu.
1000 şiir işleme alındı.


**4. Embedding ve Vektör Veritabanı (FAISS)**

Bu aşamada şiirler embedding (vektör temsili) haline getirilir ve FAISS veritabanına kaydedilir.
FAISS, metin benzerliğini hızlı hesaplamak için kullanılır.
Her şiir parçası 300 karakterlik küçük bölümlere ayrılır.

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)

docs = []
for p in poems:
    if p and isinstance(p, str) and p.strip():
        docs.extend(text_splitter.create_documents([p.strip()]))

vectorstore = FAISS.from_documents(docs, embeddings)
print("✅ FAISS veritabanı oluşturuldu!")

/tmp/ipython-input-835247746.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS veritabanı oluşturuldu!


**5. RAG Zinciri ve LLM Modeli**

Bu bölümde Gemini 2.5 Pro modeli kullanılarak LangChain tabanlı bir RAG zinciri oluşturulur.
Kullanıcının girdiği temaya göre, veri tabanındaki benzer şiirlerden esinlenerek yeni bir şiir üretilir.

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.8
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

print("Sistem hazır! Şiir üretmeye hazırsınız")


Sistem hazır! Şiir üretmeye hazırsınız


**6. Gradio Arayüzü**

Bu hücre, Gradio kullanarak etkileşimli bir web arayüzü oluşturur.
Kullanıcı şiir temasını yazar, sistem RAG modeliyle yeni bir Türkçe şiir üretir.

In [8]:
def siir_uretici(tema):
    prompt = f"""Türkçe, sanatsal ve kafiyeli bir şiir yaz.
    Şiir şu temaya uygun olmalı: '{tema}'.
    Veri tabanındaki Türkçe şiirlerin tarzından esinlen:
    Şiir özellikleri:
    - Her dize kısa olsun (4–8 kelime arası)
    - Son kelimelerde kafiye uyumu bulunsun
    - Gerekirse iç uyak (iç kafiye) da kullanılabilir
    - Akıcı, duygusal ve imgelerle dolu olsun
    - Doğa ve insan temaları kullanılabilir
    - Sadece şiiri yaz, açıklama yapma.
    """

    response = qa_chain.invoke({"question": prompt, "chat_history": []})
    return response["answer"].strip()


demo = gr.Interface(
    fn=siir_uretici,
    inputs=gr.Textbox(
        label="🎭 Şiir Teması",
        placeholder="Örnek: Ayrılık, doğa, aşk...",
        lines=1
    ),
    outputs=gr.Textbox(
        label="Üretilen Şiir",
        lines=10,
        max_lines=20
    ),
    title="Türkçe Şiir Üretici",
    theme="soft",
    allow_flagging="never"
)

demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0cd75b50111ce856b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Yeni Bölüm